# To sync or not to sync

`Estimator` and `Sampler` primitives currently only allow asynchronous calls:


In [ ]:
est = Estimator()
job = est.run([circuit], [operator], [values])
result = job.result()

In context of the algorithms, we discussed whether we want to follow the same interface for advanced subroutines like gradients or fidelities, as

In [ ]:
grad = ParameterShiftEstimatorGradient()
job = grad.run([circuit], [operator], [values])
result = job.result()

 Alternatively we would only allow synchronous calls as

In [ ]:
grad = ParameterShiftEstimatorGradient()
result = grad.evaluate([circuit], [operator], [values])

### Pro sync

* easier syntax
* "where do we draw the line?" -- if subroutines have `.run` do algorithms also need it?

### Pro async

* consistency with other estimator and sampler
* subroutines and primitives are used in the same piece of code -- different signatures look weird

### Examples

#### VQE "sync"

In [ ]:
class VQE(MinimumEigensolver):
    ...
    
    def compute_minimum_eigenvalue(self, operator):
        
        def energy(theta):
            job = self.estimator.run([self.ansatz], [operator], [theta])
            return job.result()
            
        def gradient(theta):
            return self.gradient.evaluate([self.ansatz], [operator], [theta])
        
        result = self.optimizer.minimize(energy, initial_guess, jac=gradient)

### VQE "all async"

In [ ]:
class VQE(MinimumEigensolver):
    ...
    
    def compute_minimum_eigenvalue(self, operator):
        
        def energy(theta):
            job = self.estimator.run([self.ansatz], [operator], [theta])
            return job.result()
            
        def gradient(theta):
            job = self.gradient.run([self.ansatz], [operator], [theta])
            return job.result()
        
        result = self.optimizer.minimize(energy, initial_guess, jac=gradient)



#### VQD "sync"

In [ ]:
class VQD(Eigensolver):
    ...
    
    def energy_evaluation(operator, theta, penalty):
        job = self.estimator.run([self.ansatz], [operator], [theta])
        energy = job.result()
        
        overlap = self.fidelity.evaluate([self.ansatz], [self.ansatz], [self.optimal_point], [theta])
        
        return energy + penalty * overlap

#### VQD "all async"

In [ ]:
class VQD(Eigensolver):
    ...
    
    def energy_evaluation(operator, theta, penalty):
        jobs = [
            self.estimator.run([self.ansatz], [operator], [theta]),
            self.fidelity.run([self.ansatz], [self.ansatz], [self.optimal_point], [theta])
        ]
        energy, overlap = [job.result() for job in jobs]
                
        return energy + penalty * overlap